In [ ]:
# Non-included packages
!pip install biopython

     |████████████████████████████████| 2.3 MB 4.2 MB/s 


In [ ]:
#Packages
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import nltk
import random
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
from Bio import SeqIO

#Tools
from google.colab import drive
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.python.client import device_lib
tf.test.gpu_device_name()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15263868967107073472]

In [ ]:
#Avoid warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check running in colab
try:
  from google.colab import files
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
#Configure colab vs local
if (IN_COLAB == True) :
  drive.mount('/content/drive', force_remount=True)
  root_dir = "/content/drive/My Drive/"
else:
  root_dir = "./"

## Data loading

In [ ]:
# Load hotspots
# Only hotspots of up to length 1,500 bp were taken
# Those that were shorter than 1,500 bp were padded with N's

hotspots = list(SeqIO.parse(root_dir + "Data/hotspots/fasta/" + "combined-max-1500-padded-REMOVED-BAD.fasta", "fasta"))

# These random sequences were generated from all parts of the human genome
# They were generated with the exact length profile of the hotspots
# i.e. hotspots and sequences have the same amount of padding, which is up to 1,500 bps

nohotspots = list(SeqIO.parse(root_dir + "Data/hotspots/fasta/" + "sample-max-1500-padded-REMOVED-BAD.fasta", "fasta"))

## Data Preprocessing

### Generate list of kmers

In [ ]:
k=3

def getKmers(sequence, size=5):
    return [str(sequence[x:x+size].lower()) for x in range(len(sequence) - size + 1)]

hotspots_list = []
for i, seq_record in enumerate(hotspots):
    hotspots_list.append(getKmers(seq_record.seq, size=k))
del hotspots

nohotspots_list = []
for i, seq_record in enumerate(nohotspots):
    nohotspots_list.append(getKmers(seq_record.seq, size=k))
del nohotspots

### Generate labels

In [ ]:
labels_hotspots = np.zeros(len(hotspots_list))
labels_nohotspots = np.ones(len(nohotspots_list))

### Create dataset

In [ ]:
hotspots = np.array(hotspots_list + nohotspots_list)
labels = np.concatenate((labels_hotspots, labels_nohotspots), axis=0)

In [ ]:
del hotspots_list
del nohotspots_list
del labels_hotspots
del labels_nohotspots

## Save



In [ ]:
np.save(root_dir + "Data/hotspots/kmers/hotspots-3k-list",hotspots)
np.save(root_dir + "Data/hotspots/kmers/labels_hotspots-3k-list",labels)